In [5]:
import os
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA 
from langchain.document_loaders import TextLoader 
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import DirectoryLoader, PyPDFLoader

from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI




In [3]:
OPENAI_API_KEY= (os.getenv("OPENAI_API_KEY"))

In [2]:
import json
from llama_index.readers.google import GoogleDriveReader

# Lade die Service-Account-Schlüsseldatei
with open("credentials.json") as f:
    service_account_key = json.load(f)

# Initialize the reader
reader = GoogleDriveReader(
    folder_id="1omQKaU6syLr5X0kcr1_Vnl0LKNfQzanI",
    service_account_key=service_account_key,  # Hier das geladene Dict übergeben!
)


In [3]:
new_documents = reader.load_data()

# Wenn du Duplikate vermeiden willst, kannst du einen Hash oder die Datei-ID als Schlüssel verwenden.
# Beispiel: Setzen eines Sets, um IDs zu speichern und Duplikate zu vermeiden
unique_documents = []
seen_ids = set()  # Hier speichern wir bereits gesehene IDs

for doc in new_documents:
    doc_id = getattr(doc, "doc_id", None)  # Annahme: Jedes Dokument hat eine ID
    if doc_id not in seen_ids:
        unique_documents.append(doc)
        seen_ids.add(doc_id)

# Die unique_documents-Liste enthält nun nur noch die eindeutigen Dokumente.
print(f"Anzahl eindeutiger Dokumente: {len(unique_documents)}")

Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 37 0 (offset 0)
Ignoring wrong pointing object 39 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong p

Failed to load file C:\Users\letie\AppData\Local\Temp\tmp6u3m6f7p\1SoaebZiXaU8k10roMiogiVwUAQKRW1AI.pdf with error: RetryError[<Future at 0x25bd0d6cbd0 state=finished raised PdfStreamError>]. Skipping...


Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)


Anzahl eindeutiger Dokumente: 85


In [4]:


loader = DirectoryLoader("./bendig/", glob= "./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

Ignoring wrong pointing object 10 0 (offset 0)


In [8]:
from langchain.schema import Document

# Neue Liste mit korrekten `Document`-Objekten erstellen
corrected_documents = [
    Document(page_content=doc.text_resource.text, metadata=doc.metadata) for doc in new_documents
]




In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(corrected_documents)

In [6]:
print(texts)

[Document(metadata={'source': 'bendig\\200513_Entw AbwicklV_Bendig - EBE.pdf', 'page': 0, 'page_label': '1'}, page_content='Abwicklungsvereinbarung \n \nzwischen \n \nEBE Elektro-Bau-Elemente GmbH, Sielminger Straße 63, 70771 Leinfelden-Echterdingen  \nvertreten durch den Geschäftsführer Dr. Gerd Fricke \n– Arbeitgeber – \n \nund \n \nRalf Bendig, Bachstr. 24-1 71229 Leonberg \n– Arbeitnehmer – \n \n \nPräambel \nDie Arbeitgeberin hat das mit dem Arbeitnehmer bestehende Arbeitsverhältnis vom \n01.04.2018 aus dringenden betrieblichen Gründen ordentlich am 27.04.2020 fristgerecht zum \n31.05.2020 gekündigt. \n \nDer Arbeitgeber geht davon aus, dass der Arbeitsplatz des Arbeitnehmers sp ätestens zum \nAblauf des 31. Mai 2020 vollst ändig gestrichen werden wird und somit seine \nBeschäftigungsmöglichkeit spätestens zu diesem Zeitpunkt ersatzlos wegfallen wird. Freie \noder freiwerdende Stellen, auf denen  der Arbeitnehmer mit Blick auf seine Qualifikation ab \ndem 01. Juni 2020 besch

Embed and store the embedding


In [11]:
#supplying a persist_directory will store embedding on disk
persist_directory = "db1"

## OpenAIEmbeddings
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(documents=texts, embedding=embedding, persist_directory=persist_directory)


C:\Users\letie\AppData\Local\Temp\ipykernel_15992\3519870821.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()


Make a retriever

In [31]:
retriever = vectordb.as_retriever()

corrected_documents= retriever.get_relevant_documents("Wer ist Bendig?")

In [32]:
len(corrected_documents)

4

In [14]:
retriever = vectordb.as_retriever(search_kwargs={ "k": 8})

In [15]:
retriever.search_type

'similarity'

In [16]:
retriever.search_kwargs

{'k': 8}

Make a chain

In [17]:
memory = ConversationBufferMemory(
    memory_key="chat_history", 
    return_messages=True,
    output_key="result"  # Hier wird festgelegt, dass nur "result" gespeichert wird
)


C:\Users\letie\AppData\Local\Temp\ipykernel_15992\2802410121.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [33]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

# Definiere dein individuelles Prompt-Template
custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="Beantworte die folgende Frage basierend auf dem gegebenen Kontext:\n\n"
             "Kontext:\n{context}\n\n"
             "Frage: {question}\n\n"
             "Antwort:"
)

# RetrievalQA mit dem benutzerdefinierten Prompt
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    memory=memory,
    chain_type_kwargs={"prompt": custom_prompt}  # Hier wird das Prompt eingebunden
)


In [34]:
qa_chain = RetrievalQA.from_chain_type (llm = OpenAI(),
                                       chain_type="stuff",
                                       retriever=retriever,
                                       return_source_documents=True,
                                       memory=memory)

In [35]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nQuellen:')
    for source in llm_response["source_documents"]:
        print(source.metadata.get('file path', 'Unbekannte Quelle'))


In [ ]:
query = "Was hat Bendig bekommen am Ende seines Falles ?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Am Ende seines Falles hat Bendig ein wohlwollendes, qualifiziertes Endzeugnis mit einer Führungs- und Leistungsbeurteilung mit der Note sehr gut, im Sozialverhalten "stets vorbildlich" und dem Grund des Ausscheidens "auf eigenen Wunsch" erhalten. Zudem hat er die ordnungsgemäß ausgefüllten Arbeitspapiere und eine Bescheinigung nach 312 SGB III erhalten.


Quellen:
Cases/2020-05-05_Bendig_EBE_Kuendigung/EBE Arbeitsvertrag Ralf Bendig 2017[110].pdf
Cases/2020-09-09_Meyer_Aldi_Kuendigung/200910_Protokoll Hergang Kündigung_neu.docx
Cases/2020-09-09_Meyer_Aldi_Kuendigung/20201102_Vergleichsentwurf_Anm LHL.doc
Cases/2020-09-09_Meyer_Aldi_Kuendigung/20201102_Vergleichsentwurf_V2.0.doc


: 

In [17]:

# Den Chatverlauf aus dem Memory abrufen
chat_history = memory.chat_memory.messages

# Den Verlauf formatieren und anzeigen
for message in chat_history:
    if message.type == "human":
        print(f"User: {message.content}")
    elif message.type == "ai":
        print(f"AI: {message.content}")

User: Was hat er bekommen ?
AI:  Er hat eine Abfindung von 7.650 € erhalten.


In [18]:
print(qa_chain.memory.buffer)

[HumanMessage(content='Was hat er bekommen ?', additional_kwargs={}, response_metadata={}), AIMessage(content=' Er hat eine Abfindung von 7.650 € erhalten.', additional_kwargs={}, response_metadata={})]
